### Descripción del Problema

El objetivo de este ejercicio es crear un prototipo de aplicación utilizando la API del modelo GPT-3 liberado por OpenAI. La aplicación permitirá a los usuarios interactuar con el modelo GPT-3 para generar texto a partir de una entrada dada.

### Herramientas y Servicios a Utilizar

- **OpenAI GPT-3 API:** Utilizaremos la API de OpenAI GPT-3 para generar texto.
  
- **Azure Cognitive Search:** Utilizaremos Azure Cognitive Search para indexar y buscar información dentro de documentos PDF.
  
- **Azure Blob Storage:** Almacenaremos documentos PDF relacionados con la Especialización en IA de la UAO en Azure Blob Storage para su posterior búsqueda.

### Pasos a Seguir

1. **Cargar los Documentos en Azure Blob Storage:** Subiremos documentos relacionados con la Especialización en IA de la UAO a Azure Blob Storage para que estén disponibles para el servicio de Cognitive Search.
   
2. **Configurar Azure Cognitive Search:** Configuraremos un índice en Azure Cognitive Search para indexar los documentos PDF almacenados en Azure Blob Storage.
   
3. **Interactuar con la API de GPT-3:** Utilizaremos la API de OpenAI GPT-3 para generar texto a partir de la entrada del usuario. Esto incluirá la generación de respuestas a preguntas, completar frases, y más.
   
4. **Integrar la Búsqueda en el Prototipo de Aplicación:** Implementaremos una función en el prototipo de aplicación que permita a los usuarios buscar información dentro de los documentos PDF utilizando Azure Cognitive Search.
   
5. **Evaluar el Modelo GPT-3 Turbo y DALLE-3:** Probaremos tanto el modelo GPT-3 Turbo como el modelo DALLE-3 para evaluar su desempeño y determinar cuál es más adecuado para nuestras necesidades.


In [0]:
import os
import openai
import dotenv

# Función para cargar las variables de entorno desde el archivo .env
def load_environment_variables():
    dotenv.load_dotenv()
    return (
        os.getenv("AZURE_OPENAI_ENDPOINT"),  # Endpoint de la API de OpenAI
        os.getenv("AZURE_OPENAI_API_KEY"),   # Clave de la API de OpenAI
        os.getenv("SEARCH_ENDPOINT"),        # Endpoint de Azure Cognitive Search
        os.getenv("SEARCH_KEY"),             # Clave de Azure Cognitive Search
        os.getenv("SEARCH_INDEX_NAME")       # Nombre del índice en Azure Cognitive Search
    )

In [0]:
def run_openai_completion(content):
    # Cargar las variables de entorno
    endpoint, api_key, search_endpoint, search_key, search_index = load_environment_variables()
    
    # Nombre del despliegue de OpenAI
    deployment = "Taller_gpt"

    # Crear un cliente para interactuar con la API de OpenAI
    client = openai.AzureOpenAI(
        base_url=f"{endpoint}/openai/deployments/{deployment}/extensions",  # URL base del despliegue de OpenAI
        api_key=api_key,  # Clave de la API de OpenAI
        api_version="2023-09-01-preview"  # Versión de la API
    )

    # Crear una solicitud de completado utilizando el modelo GPT-3
    completion = client.chat.completions.create(
        messages=[{"role": "user", "content": content}],  # Mensaje del usuario
        model=deployment,  # Nombre del modelo GPT-3 a utilizar
        extra_body={
            "dataSources": [
                {
                    "type": "AzureCognitiveSearch",  # Tipo de fuente de datos (Azure Cognitive Search)
                    "parameters": {
                        "endpoint": search_endpoint,    # Endpoint de Azure Cognitive Search
                        "key": search_key,              # Clave de Azure Cognitive Search
                        "indexName": search_index,      # Nombre del índice en Azure Cognitive Search
                        "roleInformation": "Eres un asistente de IA que ayuda con información de la Especialización en IA de la UAO",  # Información sobre el rol del asistente
                        "semanticConfiguration": "default",  # Configuración semántica
                        "queryType": "simple",          # Tipo de consulta
                        "fieldsMapping": {},            # Mapeo de campos
                        "inScope": True,                # Si está en el ámbito de búsqueda
                        "strictness": 3,                # Rigurosidad de la búsqueda
                        "topNDocuments": 5,             # Número máximo de documentos a devolver
                    }
                }
            ]
        }
    )
    
    # Retornar la respuesta generada por GPT-3
    return f"{completion.choices[0].message.content}"


In [0]:
# Ejemplo de uso:
content = "Quien es el coordinador de la especialización?"
response = run_openai_completion(content)
print(response)

In [0]:
def chat():
    print("Bienvenido al Chat con el Asistente IA\n")
    print("Puedes escribir tus mensajes a continuación. Escribe 'salir' para terminar.\n")

    while True:
        user_input = input("Tú: ")
        print("Usuario:", user_input)

        if user_input.lower() == "salir":
            print("Hasta luego!")
            break

        response = run_openai_completion(user_input)
        print("Asistente IA:", response)

if __name__ == "__main__":
    chat()

In [0]:
# Importar las bibliotecas necesarias
import os
from openai import AzureOpenAI
import json

# Crear un cliente para interactuar con la API de OpenAI
client = AzureOpenAI(
    api_version="2024-02-01",  # Versión de la API
    azure_endpoint="https://base-ia-openai.openai.azure.com/",  # Endpoint de la API de Azure OpenAI
    api_key=os.environ["AZURE_OPENAI_API_KEY"],  # Clave de la API de OpenAI
)

# Generar una imagen utilizando DALL-E 3
result = client.images.generate(
    model="Dalle3",  # Nombre del modelo DALL-E 3
    prompt="Creame un avatar con una mezcla de ave rapaz y un ser humano",  # Prompt para la generación de la imagen
    n=1  # Número de imágenes a generar
)

# Obtener la URL de la imagen generada
image_url = json.loads(result.model_dump_json())['data'][0]['url']


In [0]:
import json
import requests
from PIL import Image
from io import BytesIO
from openai import AzureOpenAI

# Descargar la imagen utilizando su URL
response = requests.get(image_url)

# Abrir la imagen descargada utilizando PIL (Python Imaging Library)
image = Image.open(BytesIO(response.content))

# Mostrar la imagen en una ventana emergente
image.show()


In [0]:
# Nombre del despliegue de OpenAI para embeddings
deployment = "Embed"

# Crear embeddings utilizando el modelo especificado
embeddings = client.embeddings.create(
    model=deployment,  # Nombre del modelo
    input="The food was delicious and the waiter..."  # Texto de entrada para generar embeddings
)

# Imprimir los embeddings generados
print(embeddings)

## Conclusiones Técnicas:

1. **Integración de OpenAI GPT-3 y Azure Cognitive Search:** La combinación de la API de OpenAI GPT-3 y Azure Cognitive Search permite una integración efectiva entre generación de texto y búsqueda de información en documentos PDF. Esto proporciona a los usuarios una experiencia más completa y enriquecedora al interactuar con la aplicación.

2. **Configuración de Azure Cognitive Search:** La configuración de Azure Cognitive Search para indexar y buscar información dentro de documentos PDF facilita la recuperación de datos relevantes para el usuario. La configuración adecuada de los parámetros, como el tipo de consulta y el número máximo de documentos a devolver, es crucial para obtener resultados precisos.

3. **Uso de modelos de lenguaje avanzados:** La utilización de modelos de lenguaje avanzados como GPT-3 y DALLE-3 permite generar texto de manera autónoma y responder a las consultas de los usuarios de manera más natural y efectiva.

4. **Pruebas y evaluación de modelos:** Es importante realizar pruebas y evaluaciones exhaustivas de los modelos GPT-3 y DALLE-3 para determinar cuál es más adecuado para las necesidades específicas de la aplicación. Esto implica evaluar la calidad de las respuestas generadas, así como el rendimiento y la eficiencia de cada modelo.

## Conclusiones Generales:

1. **Capacidad de generación de texto:** La aplicación permite a los usuarios generar texto de manera libre y obtener respuestas pertinentes a sus consultas. Esto facilita la interacción con la información y puede ser útil en una variedad de contextos, desde la investigación académica hasta la resolución de problemas prácticos.

2. **Facilidad de acceso a la información:** La integración con Azure Cognitive Search permite a los usuarios buscar información específica dentro de documentos PDF relacionados con la Especialización en IA de la UAO de manera rápida y eficiente. Esto mejora la accesibilidad a la información y facilita la toma de decisiones informadas.

3. **Aplicabilidad en diversos escenarios:** La aplicación puede ser útil en una variedad de escenarios, incluyendo la educación, la investigación y la industria. La capacidad de generar texto y buscar información de manera inteligente puede mejorar la productividad y la eficiencia en diferentes contextos.

4. **Potencial de mejora continua:** Aunque el prototipo de la aplicación funciona correctamente, siempre hay margen para mejoras adicionales. Esto puede incluir la optimización de los modelos de lenguaje, la mejora de la precisión de la búsqueda y la expansión de las funcionalidades ofrecidas a los usuarios.
